# Down-scaled multi-area model

## Import dependencies

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
!pip install nested_dict dicthash

In [7]:
%%html
<style>
table {float:left}
</style>

## Create config file

In [ ]:
with open('config.py', 'w') as fp:
    fp.write(
'''import os
base_path = os.path.abspath(".")
data_path = os.path.abspath("simulations")
jobscript_template = "python {base_path}/run_simulation.py {label}"
submit_cmd = "bash -c"
''')

In [ ]:
from multiarea_model import MultiAreaModel

## Specify paramters of model

### 1. Scaling parameter
**Scaling parameter** (scale_down_to) is the parameter defines the the ratio of the full scale multi-area model being down-scaled to a model with fewer neurons and indegrees so as to be simulated on machines with lower computational ability and the simulation results can be obtained in relative shorter period of time.<br> <br> 
Neurons and indegrees are both scaled down to 0.5%, where the model can usually be simulated on a local machine.<br> **Warning: This will not yield reasonable dynamical results from the network and is only meant to demonstrate the simulation workflow.**

In [ ]:
scale_down_to = 0.005

### 2. Model and simulation parameters

#### 2.1 Connection parameters (conn_params)

| Parameter | Parameter description | Variable                    | Value              | Value description |
|:---------:|:----------------------|:---------------------------:|:------------------:|:------------------|
|           |                       | replace_non_simulated_areas | 'het_poisson_stat' |                   |
|           |                       | g                           | -11.               |                   |
|           |                       | K_stable                    | 'K_stable.npy'     |                   |
|           |                       | fac_nu_ext_TH               | 1.2                |                   |
|           |                       | fac_nu_ext_5E               | 1.125              |                   |
|           |                       | fac_nu_ext_6E               | 1.41666667         |                   |
|           |                       | av_indegree_V1              | 3950.              |                   |

In [ ]:
conn_params = {'replace_non_simulated_areas': 'het_poisson_stat',
               'g': -11.,
               
               'K_stable': 'K_stable.npy',
               'fac_nu_ext_TH': 1.2,
               'fac_nu_ext_5E': 1.125,
               'fac_nu_ext_6E': 1.41666667,
               'av_indegree_V1': 3950.}

#### 2.2 Input parameters (input_params)

| Parameter | Parameter description | Variable                      | Value                | Value description |
|:---------:|:----------------------|:-----------------------------:|:--------------------:|:------------------|
|           |                       | rate_ext                      |      10.             |                   |

In [ ]:
input_params = {'rate_ext': 10.}

#### 2.3 Neuron parameters (neuron_params)

| Parameter | Parameter description | Variable                    | Value              | Value description |
|:---------:|:----------------------|:---------------------------:|:------------------:|:------------------|
|           |                       | V0_mean                     | -150.              |                   |
|           |                       | V0_sd                       | 50.                |                   |

In [ ]:
neuron_params = {'V0_mean': -150.,
                 'V0_sd': 50.}

#### 2.4 Network parameters (network_params)

| Parameter | Parameter description | Variable              | Value                         | Value description |
|:---------:|:----------------------|:---------------------:|:-----------------------------:|:------------------|
|           |                       | N_scaling             | scale_down_to                 |                   |
|           |                       | K_scaling             | scale_down_to                 |                   |
|           |                       | fullscale_rates       | 'tests/fullscale_rates.json'  |                   |
|           |                       | input_params          | input_params                  |                   |
|           |                       | connection_params     | conn_params                   |                   |
|           |                       | neuron_params         | neuron_params                 |                   |

In [ ]:
network_params = {'N_scaling': scale_down_to,
                  'K_scaling': scale_down_to,
                  'fullscale_rates': 'tests/fullscale_rates.json',
                  'input_params': input_params,
                  'connection_params': conn_params,
                  'neuron_params': neuron_params}

### 3. Simulation paramters (sim_params)

| Parameter | Parameter description | Variable             | Value              | Value description |
|:---------:|:----------------------|:--------------------:|:------------------:|:------------------|
|           |                       |t_sim                 |2000.               |                   |
|           |                       |num_processes         |1                   |                   |
|           |                       |local_num_threads     |1                   |                   |
|           |                       |recording_dict        |input_params        |                   |
|           |                       |record_vm             |False               |                   |

In [ ]:
sim_params = {'t_sim': 2000.,
              'num_processes': 1,
              'local_num_threads': 1,
              'recording_dict': {'record_vm': False}}

### 4. Theory paramters (theory_params)

| Parameter | Parameter description | Variable              | Value                         | Value description |
|:---------:|:----------------------|:---------------------:|:-----------------------------:|:------------------|
|           |                       | dt                    | 0.1                           |                   |

In [ ]:
theory_params = {'dt': 0.1}

## Instantiate a multi-area model, predict firing rates from theroy 

In [ ]:
M = MultiAreaModel(network_params, simulation=True,
                   sim_spec=sim_params,
                   theory=True,
                   theory_spec=theory_params)

In [ ]:
p, r = M.theory.integrate_siegert()
print("Mean-field theory predicts an average "
      "rate of {0:.3f} spikes/s across all populations.".format(np.mean(r[:, -1])))

## Run the simulation

In [ ]:
M.simulation.simulate()

## Load and process simulation results data

### 1. Load spike data

In [ ]:
data = np.loadtxt(M.simulation.data_dir + '/recordings/' + M.simulation.label + "-spikes-1-0.dat", skiprows=3)

### 2. Compute instantaneous rate per neuron across all populations

In [ ]:
tsteps, spikecount = np.unique(data[:,1], return_counts=True)
rate = spikecount / M.simulation.params['dt'] * 1e3 / np.sum(M.N_vec)

## Simulation results visualization

### 1. Instantaneous and mean rate

In [ ]:
fig, ax = plt.subplots()
ax.plot(tsteps, rate)
ax.plot(tsteps, np.average(rate)*np.ones(len(tsteps)), label='mean')
ax.set_title('instantaneous rate across all populations')
ax.set_xlabel('time (ms)')
ax.set_ylabel('rate (spikes / s)')
ax.set_xlim(0, sim_params['t_sim'])
ax.set_ylim(0, 50)
ax.legend()